<a href="https://colab.research.google.com/github/Kakaoko/Sandwich-Panel-Investigation/blob/master/Sandwich_Panel_Transfer_Matrix_Modelling(Jupyter_Ver_).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **< Sandwich Panel Transfer Matrix Modelling >**

*   Sandwich Panel에 성능을 파악하기 위해 Transfer Matrix Method를 사용하여 모델링 하였습니다.

*   첫 번째 모델은 Panel -> Air -> Porous -> Air -> Panel을 거쳐 소음이 전달됩니다.
*   두 번째 모델은 Panel -> Air -> Porous -> Shock Absorber -> Porous -> Air -> Panel을 거쳐 소음이 전달됩니다.



**(해당 코드를 먼저 실행해주십시오.)**

In [0]:
from ipywidgets import interact
import numpy as np
import matplotlib.pyplot as plt

#Material property
#PLA for 3D Printing
global ela_pla, den_pla, c_pla, c_air, zc_air
e_pla = 3.5 * 10**9
d_pla = 1240
c_pla = 2220
c_air = 344
zc_air = 409.4

#Thin Elastic Panel Transfer Matrix
class panel :
    def __init__(self, f, b, h, L, theta) :
        self.f = f
        self.b = b
        self.h = h
        self.L = L
        self.theta = theta

    def w(self) :
        return 2 * np.pi * self.f

    def k_pla(self) :
        return 2 * np.pi * self.f / c_pla

    def k_air(self) :
        return 2 * np.pi * self.f / c_air

    def B(self) :
        return e_pla * self.b * self.h**3 / (12 * self.L)

    def Zp(self) :
        return panel(self.f, self.b, self.h, self.L, self.theta).B() * (panel(self.f, self.b, self.h, self.L, self.theta).k_air()**4 * np.sin(self.theta)**4 - panel(self.f, self.b, self.h, self.L, self.theta).k_pla()**4) / (panel(self.f, self.b, self.h, self.L, self.theta).w() * 1j)

    def T_panel(self) :
        return np.array([[1, panel(self.f, self.b, self.h, self.L, self.theta).Zp()], [0 ,1]])

#Fluid Layer Transfer Matrix
class fluid :
    def __init__(self, f, d, theta) :
        self.f = f
        self.d = d
        self.theta = theta

    def w(self) :
        return 2 * np.pi * self.f

    def k_air(self) :
        return 2 * np.pi * self.f / c_air

    def m00(self) :
        return np.cosh(fluid(self.f, self.d, self.theta).k_air() * self.d * np.cos(self.theta) * 1j)

    def m01(self) :
        return zc_air * np.sinh(fluid(self.f, self.d, self.theta).k_air() * self.d * np.cos(self.theta) * 1j) / np.cos(self.theta)

    def m10(self) :
        return (np.sin(fluid(self.f, self.d, self.theta).k_air() * self.d * np.cos(self.theta) * 1j)) * np.cos(self.theta) / zc_air

    def m11(self) :
        return fluid(self.f, self.d, self.theta).m00()

    def T_fluid(self) :
        return np.array([[fluid(self.f, self.d, self.theta).m00(), fluid(self.f, self.d, self.theta).m01()], [fluid(self.f, self.d, self.theta).m10(), fluid(self.f, self.d, self.theta).m11()]])

#Porous Layer Transfer Matrix
class porous :
    def __init__(self, f, d, b, h, sigma, theta) :
        self.f = f
        self.d = d
        self.b = b
        self.h = h
        self.sigma = sigma
        self.theta = theta

    def w(self) :
        return 2 * np.pi * self.f

    def k_air(self) :
        return 2 * np.pi * self.f / c_air

    def mass(self) :
        return b * h * d * den_pla * (1 - sigma)

    def m00(self) :
        return np.cosh(porous(self.f, self.d, self.b, self.h, self.sigma, self.theta).k_air() * self.d * np.cos(self.theta) * 1j)

    def m01(self) :
        return zc_air * np.sinh(porous(self.f, self.d, self.b, self.h, self.sigma, self.theta).k_air() * self.d * np.cos(self.theta) * 1j) / (np.cos(self.theta) * self.sigma)

    def m10(self) :
        return (np.sin(porous(self.f, self.d, self.b, self.h, self.sigma, self.theta).k_air() * self.d * np.cos(self.theta) * 1j)) * np.cos(self.theta) * self.sigma / zc_air

    def m11(self) :
        return porous(self.f, self.d, self.b, self.h, self.sigma, self.theta).m00()

    def T_porous(self) :
        return np.array([[porous(self.f, self.d, self.b, self.h, self.sigma, self.theta).m00(), porous(self.f, self.d, self.b, self.h, self.sigma, self.theta).m01()], [porous(self.f, self.d, self.b, self.h, self.sigma, self.theta).m10(), porous(self.f, self.d, self.b, self.h, self.sigma, self.theta).m11()]])

    def T_mass(self) :
        return np.array([[1, porous(self.f, self.d, self.b, self.h, self.sigma, self.theta).w() * porous(self.f, self.d, self.b, self.h, self.sigma, self.theta).mass() * 1j], [0, 1]])

#Shock Absorber Transfer Matrix
class shock_ab :
    def __init__(self, f, c, k, theta) :
        self.f = f
        self.c = c
        self.k = k
        self.theta = theta

    def w(self) :
        return 2 * np.pi * self.f

    def m10(self) :
        return (shock_ab(self.f, self.c, self.k, self.theta).w() * 1j) / (self.k + shock_ab(self.f, self.c, self.k, self.theta).w() * self.c * 1j)

    def T_shock_ab(self) :
        return np.array([[1, 0], [shock_ab(self.f, self.c, self.k, self.theta).m10(), 1]])

해당 모델링에 사용된 파라미터는 아래의 그림과 같습니다.

*   [Panel -> Air -> Porous -> Air -> Panel](https://drive.google.com/open?id=1q2T6U9xodelsDcUdUCgVFRkum8rZO3Ai)
*   [Panel -> Air -> Porous -> Shock Absorber -> Porous -> Air -> Panel](https://drive.google.com/open?id=1SrPMWCiKJK1j5OKJq-Dsrw0DO8zDW209)

**(해당 코드를 진행하면 파라미터를 유동적으로 조절할 수 있습니다.)**

In [0]:
#Parameter Input
#Panel -> Air -> Porous -> Air -> Panel
#Panel -> Air -> Porous -> Shock Absorber -> Porous -> Panel
@interact(b = (0, 0.5, 0.05), h = (0, 0.5, 0.05), L_nor = (0, 0.1, 0.01), L_ck = (0, 0.1, 0.01), d_air = (0, 0.1, 0.01), d_p = (0, 0.1, 0.01), d_p1 = (0, 0.1, 0.01), d_p2 = (0, 0.1, 0.01), c = (0, 1000, 5), k = (0, 1000, 5), sigma = (0.01, 1, 0.01), theta = (0, np.pi / 2, np.pi / 180))
def para_input(b, h, L_nor, L_ck, d_air, d_p, d_p1, d_p2, c, k, sigma, theta) :
    tau_1, tau_2 = [], []
    TL_1, TL_2 = [], []

    Area, Area_rec = np.array([[b * h, 0], [0, 1]]), np.array([[1 / (b * h), 0], [0, 1]])

    for f in range(50, 1600) :
        T_1 = np.matmul(np.matmul(np.matmul(panel(f, b, h, L_nor, theta).T_panel(), fluid(f, d_air, theta).T_fluid()), np.matmul(porous(f, d_p, b, h, sigma, theta).T_porous(), fluid(f, d_air, theta).T_fluid())), panel(f, b, h, L_nor, theta).T_panel())
        tau_1.append(4 * np.absolute(T_1[0][0] + T_1[0][1] / zc_air + zc_air * T_1[1][0] + T_1[1][1])**-2)
        T_2 = np.matmul(np.matmul(np.matmul(np.matmul(np.matmul(panel(f, b, h, L_ck, theta).T_panel(), fluid(f, d_air, theta).T_fluid()), np.matmul(porous(f, d_p1, b, h, sigma, theta).T_porous(), Area)), np.matmul(shock_ab(f, c, k, theta).T_shock_ab(), Area_rec)), np.matmul(porous(f, d_p2, b, h, sigma, theta).T_porous(), fluid(f, d_air, theta).T_fluid())), panel(f, b, h, L_ck, theta).T_panel())
        tau_2.append(4 * np.absolute(T_2[0][0] + T_2[0][1] / zc_air + zc_air * T_2[1][0] + T_2[1][1])**-2)
              
    for i in range(len(tau_1)) :
        TL_1.append(- 10 * np.log10(tau_1[i]))
        TL_2.append(-10 * np.log10(tau_2[i]))
        
    plt.plot(np.arange(50, 1600), TL_1, ":", label = "Typical Sandwich Panel")
    plt.plot(np.arange(50, 1600), TL_2, "--", label = "Sandwich Panel with Shock Absorber")
    plt.title("Sandwich Panel Transmission Loss(dB)")
    plt.xlabel("Frequency(Hz)")
    plt.ylabel("TL(dB)")
    plt.legend(loc = "lower right")
    return plt.show()